In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-



u"""QGIS Unit tests for QgsLayout

A simple demonstration of to generate a PDF using a QGIS project
and a QGIS layout template.

Inspiration originale, en version 2, mais adaptée à la version 3
http://kartoza.com/how-to-create-a-qgis-pdf-report-with-a-few-lines-of-python/

Inspiration suivante, en version 3, mais adaptée à linux
https://github.com/carey136/Standalone-Export-Atlas-QGIS3
https://github.com/carey136/Standalone-Export-Atlas-QGIS3/blob/master/AtlasExport.py

"""


__author__ = 'Frédéric Garel'
__date__ = '02/04/2019'
__copyright__ = 'Copyright 2019'
# This will get replaced with a git SHA1 when you do a git archive
__revision__ = '$Format:%H$'

import qgis  # NOQA

import os
from os.path import expanduser
#from qgis.testing import start_app
from qgis.core import (QgsApplication,
                       QgsLayout,
                       QgsLayoutAtlas,
                       QgsLayoutExporter,
                       QgsProject,
                       QgsPrintLayout)



class Imprimeur(object):
    u"""
    Classe d'objet dont l'objectif est de fournir un Imprimeur
    c'est à dire, un objet capable de generer des cartes, des manipulations
    """

    def __init__(self, dictionnaire_parametres):
        u"""
        Méthode pour initialiser l'objet.

        Lorque l'objet est instancié, c'est cette méthode qui est executée.

        """
        self.aecrire = ''
        self.dictionnaire_parametres =  dictionnaire_parametres
        print("{}".format(self.dictionnaire_parametres[myProject]))

        

    def make_simple_pdf(self):

        #### Initialising QGIS in back end (utilising users temp folder) ####
        # Il exsite 2 facons de lancer qgis en mode autonome :
        # - 1ère methode de lancement (mode production)
        #   create a reference to the QgsApplication, setting the
        #   second argument for GUI
        #     False = disables the GUI = standalone scripts
        #     True = enable the GUI = custom application
        gui_flag = False
        app = QgsApplication([], gui_flag)
        #   Make sure QGIS_PREFIX_PATH is set in your env if needed!
        app.initQgis()

        # - 2de methode de lancement (mode debugage)
        #   start_app()

        # Une instance de QGis va ouvrir le projet
        projectInstance = QgsProject.instance()
        # project_path = '/home/fred/Documents/report/source/QGisEnCoulisse/data/dtdict2.qgz'
        projectInstance.read(myProject)
        # print(project.fileName())

        # Le gestionnaire de mise en page est utilisé pour ouvrir notre layout
        #layout_name = 'A3_Paysage'
        layoutManager = projectInstance.layoutManager()
        layout = layoutManager.layoutByName(layoutName) #Layout name
        #print(layout.items()[1])

        layoutExporter = QgsLayoutExporter(layout)
        settings = QgsLayoutExporter.PdfExportSettings()
        settings.dpi = 80
        settings.rasterizeWholeImage = False
        settings.forceVectorOutput = False
        #settings.exportMetadata = True
        #print(settings.dpi)
        resultpdf_path = '/home/fred/Documents/report/source/QGisEnCoulisse/data/report.pdf'
        resultpdf_path = '/home/fred/f/CARTOGRAPHIE/donnees/urbanisme/NUMEROTATIONS/2018/tadebugnumero/report.pdf'
        layoutExporter.exportToPdf(resultpdf_path, settings)

        projectInstance.clear()



    def make_atlas_pdf(self):

        #### Initialising QGIS in back end (utilising users temp folder) ####
        #home = expanduser("~")
        #QgsApplication( [], False, home + "/.local/tmp" )
        #QgsApplication.setPrefixPath("/usr/bin/qgis", True) #Change path for standalone QGIS install
        gui_flag = False
        app = QgsApplication([], gui_flag)
        #   Make sure QGIS_PREFIX_PATH is set in your env if needed!
        app.initQgis()


        #### Defining map path and contents ####
        project = QgsProject.instance()
        manager = project.layoutManager()
        print('avant chargement myProject = {}'.format(myProject))
        project.read(myProject)
        print('après chargement myProject = {}'.format(myProject))
        # Le gestionnaire de mise en page est utilisé pour ouvrir notre layout
        myLayout = manager.layoutByName(layoutName) #Layout name
        print('layoutName = {}'.format(layoutName))
        myAtlas = myLayout.atlas()
        myAtlasMap = myAtlas.layout()

        #### atlas query ####
        if(coverageLayer in locals()):
            myAtlas.setCoverageLayer(QgsProject.instance().mapLayersByName(coverageLayer))
        print('coverageLayer = {}'.format(coverageLayer))
        myAtlas.setFilterFeatures(True)
        myAtlas.setFilterExpression(atlasFilter)
        print('atlasFilter = {}'.format(atlasFilter))

        #### image output name ####
        myAtlas.setFilenameExpression(outputName)
        print('outputName = {}'.format(outputName))

        #### image and pdf settings ####
        pdf_settings=QgsLayoutExporter(myAtlasMap).PdfExportSettings()
        image_settings = QgsLayoutExporter(myAtlasMap).ImageExportSettings()
        image_settings.dpi = 200

        #### Export images or PDF (depending on flag) ####
        if outputFormat == "image":
            imageExtension = '.jpg'
            for myLayout in QgsProject.instance().layoutManager().printLayouts():
                print('myLayout = {}'.format(myLayout))
                if myAtlas.enabled():
                    result, error = QgsLayoutExporter.exportToImage(myAtlas,
                                        baseFilePath=outputFolder + '/', extension=imageExtension, settings=image_settings)
                    if not result == QgsLayoutExporter.Success:
                        print(error)
        if outputFormat == "pdf":
            for myLayout in QgsProject.instance().layoutManager().printLayouts():
                print('myLayout = {}'.format(myLayout))
                if myAtlas.enabled():
                    result, error = QgsLayoutExporter.exportToPdf(myAtlas, outputFolder + '/' + pdfName + '.pdf', settings=pdf_settings)
                    if not result == QgsLayoutExporter.Success:
                        print(error)
        print("Success!")


def main():
    u"""
    Programme principal.

    Ce script peut etre :
    - autonome : il s'utilise alors ainsi
      python3 ./espu_voirieAdministratif_qgis_python_printing.py

    """

    # On defintit nos variables dans un dictionnaire
    #   1. myProject     = le nom du fichier projet Qgis
    #   2. layoutName    = le nom de la mise en page
    #   3. coverageLayer = le nom de l'entité couche de couverture
    #   4. atlasFilter   = Filtre de l'atlas
    #   5. outputFormat  = Format du fichier sortie
    #   6. outputFolder  = le repertoire de sortie
    #   7. outputName    = On coche l'option pour créer un seul fichier pdf en résultat
    #   8. pdfName       = le nom du fichier resultat
    dico_tadebugnumero={
        myProject: '/home/fred/Documents/technic/source/voirie/data/numerotations_linux.qgz',
        layoutName: 'TADebugNumero',
        coverageLayer: 'lj_demande_numero_parcelle',
        atlasFilter: '',
        outputFormat: "pdf",
        outputFolder: '/home/fred/f/CARTOGRAPHIE/donnees/urbanisme/NUMEROTATIONS/2018/tadebugnumero',
        outputName: "query producing unique value",
        pdfName: 'report'
    }
    __myImprimeur = Imprimeur(dico_tadebugnumero)
    print("{}".format(dico_tadebugnumero[myProject]))
    #make_atlas_pdf()


if __name__ == '__main__':
    main()
    #pass


/home/fred/Documents/technic/source/voirie/data/numerotations_linux.qgz
/home/fred/Documents/technic/source/voirie/data/numerotations_linux.qgz


In [18]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-



u"""QGIS Unit tests for QgsLayout

A simple demonstration of to generate a PDF using a QGIS project
and a QGIS layout template.

Inspiration originale, en version 2, mais adaptée à la version 3
http://kartoza.com/how-to-create-a-qgis-pdf-report-with-a-few-lines-of-python/

Inspiration suivante, en version 3, mais adaptée à linux
https://github.com/carey136/Standalone-Export-Atlas-QGIS3
https://github.com/carey136/Standalone-Export-Atlas-QGIS3/blob/master/AtlasExport.py

"""


__author__ = 'Frédéric Garel'
__date__ = '02/04/2019'
__copyright__ = 'Copyright 2019'
# This will get replaced with a git SHA1 when you do a git archive
__revision__ = '$Format:%H$'

import qgis  # NOQA

import os
from os.path import expanduser
#from qgis.testing import start_app
from qgis.core import (QgsApplication,
                       QgsLayout,
                       QgsLayoutAtlas,
                       QgsLayoutExporter,
                       QgsProject,
                       QgsPrintLayout)


class Imprimeur(object):
    u"""
    Classe d'objet dont l'objectif est de fournir un Imprimeur
    c'est à dire, un objet capable de generer des cartes, des manipulations
    """

    def __init__(self, dictionnaire_parametres):
        u"""
        Méthode pour initialiser l'objet.

        Lorque l'objet est instancié, c'est cette méthode qui est executée.

        """
        self.aecrire = ''
        self.dictionnaire_parametres =  dictionnaire_parametres
        print("{}".format(self.dictionnaire_parametres['myProject']))

        

    def make_simple_pdf(self):

        #### Initialising QGIS in back end (utilising users temp folder) ####
        # Il exsite 2 facons de lancer qgis en mode autonome :
        # - 1ère methode de lancement (mode production)
        #   create a reference to the QgsApplication, setting the
        #   second argument for GUI
        #     False = disables the GUI = standalone scripts
        #     True = enable the GUI = custom application
        gui_flag = False
        app = QgsApplication([], gui_flag)
        #   Make sure QGIS_PREFIX_PATH is set in your env if needed!
        app.initQgis()

        # - 2de methode de lancement (mode debugage)
        #   start_app()

        # Une instance de QGis va ouvrir le projet
        projectInstance = QgsProject.instance()
        # project_path = '/home/fred/Documents/report/source/QGisEnCoulisse/data/dtdict2.qgz'
        projectInstance.read(myProject)
        # print(project.fileName())

        # Le gestionnaire de mise en page est utilisé pour ouvrir notre layout
        #layout_name = 'A3_Paysage'
        layoutManager = projectInstance.layoutManager()
        layout = layoutManager.layoutByName(layoutName) #Layout name
        #print(layout.items()[1])

        layoutExporter = QgsLayoutExporter(layout)
        settings = QgsLayoutExporter.PdfExportSettings()
        settings.dpi = 80
        settings.rasterizeWholeImage = False
        settings.forceVectorOutput = False
        #settings.exportMetadata = True
        #print(settings.dpi)
        resultpdf_path = '/home/fred/Documents/report/source/QGisEnCoulisse/data/report.pdf'
        resultpdf_path = '/home/fred/f/CARTOGRAPHIE/donnees/urbanisme/NUMEROTATIONS/2018/tadebugnumero/report.pdf'
        layoutExporter.exportToPdf(resultpdf_path, settings)

        projectInstance.clear()



    def make_atlas_pdf(self):

        #### Initialising QGIS in back end (utilising users temp folder) ####
        #home = expanduser("~")
        #QgsApplication( [], False, home + "/.local/tmp" )
        #QgsApplication.setPrefixPath("/usr/bin/qgis", True) #Change path for standalone QGIS install
        gui_flag = False
        app = QgsApplication([], gui_flag)
        #   Make sure QGIS_PREFIX_PATH is set in your env if needed!
        app.initQgis()


        #### Defining map path and contents ####
        project = QgsProject.instance()
        manager = project.layoutManager()
        print('avant chargement myProject = {}'.format(myProject))
        project.read(myProject)
        print('après chargement myProject = {}'.format(myProject))
        # Le gestionnaire de mise en page est utilisé pour ouvrir notre layout
        myLayout = manager.layoutByName(layoutName) #Layout name
        print('layoutName = {}'.format(layoutName))
        myAtlas = myLayout.atlas()
        myAtlasMap = myAtlas.layout()

        #### atlas query ####
        if(coverageLayer in locals()):
            myAtlas.setCoverageLayer(QgsProject.instance().mapLayersByName(coverageLayer))
        print('coverageLayer = {}'.format(coverageLayer))
        myAtlas.setFilterFeatures(True)
        myAtlas.setFilterExpression(atlasFilter)
        print('atlasFilter = {}'.format(atlasFilter))

        #### image output name ####
        myAtlas.setFilenameExpression(outputName)
        print('outputName = {}'.format(outputName))

        #### image and pdf settings ####
        pdf_settings=QgsLayoutExporter(myAtlasMap).PdfExportSettings()
        image_settings = QgsLayoutExporter(myAtlasMap).ImageExportSettings()
        image_settings.dpi = 200

        #### Export images or PDF (depending on flag) ####
        if outputFormat == "image":
            imageExtension = '.jpg'
            for myLayout in QgsProject.instance().layoutManager().printLayouts():
                print('myLayout = {}'.format(myLayout))
                if myAtlas.enabled():
                    result, error = QgsLayoutExporter.exportToImage(myAtlas,
                                        baseFilePath=outputFolder + '/', extension=imageExtension, settings=image_settings)
                    if not result == QgsLayoutExporter.Success:
                        print(error)
        if outputFormat == "pdf":
            for myLayout in QgsProject.instance().layoutManager().printLayouts():
                print('myLayout = {}'.format(myLayout))
                #if myAtlas.enabled():
                #    result, error = QgsLayoutExporter.exportToPdf(myAtlas, outputFolder + '/' + pdfName + '.pdf', settings=pdf_settings)
                #    if not result == QgsLayoutExporter.Success:
                #        print(error)
        print("Success!")


def main():
    u"""
    Programme principal.

    Ce script peut etre :
    - autonome : il s'utilise alors ainsi
      python3 ./espu_voirieAdministratif_qgis_python_printing.py

    """
    myProject = ''
    layoutName = ''
    coverageLayer = ''
    atlasFilter = ''
    outputFormat = ""
    outputFolder = ''
    outputName = ""
    pdfName =  ''
    # On defintit nos variables dans un dictionnaire
    #   1. myProject     = le nom du fichier projet Qgis
    #   2. layoutName    = le nom de la mise en page
    #   3. coverageLayer = le nom de l'entité couche de couverture
    #   4. atlasFilter   = Filtre de l'atlas
    #   5. outputFormat  = Format du fichier sortie
    #   6. outputFolder  = le repertoire de sortie
    #   7. outputName    = On coche l'option pour créer un seul fichier pdf en résultat
    #   8. pdfName       = le nom du fichier resultat
    dico_tadebugnumero={
        'myProject': '/home/fred/Documents/technic/source/voirie/data/numerotations_linux.qgz',
        'layoutName': 'TADebugNumero',
        'coverageLayer': 'lj_demande_numero_parcelle',
        'atlasFilter': '',
        'outputFormat': "pdf",
        'outputFolder': '/home/fred/f/CARTOGRAPHIE/donnees/urbanisme/NUMEROTATIONS/2018/tadebugnumero',
        'outputName': "query producing unique value",
        'pdfName': 'report'
    }
    __myImprimeur = Imprimeur(dico_tadebugnumero)
    print("{}".format(dico_tadebugnumero['myProject']))
    __myImprimeur.make_atlas_pdf()


if __name__ == '__main__':
    main()
    #pass


/home/fred/Documents/technic/source/voirie/data/numerotations_linux.qgz
/home/fred/Documents/technic/source/voirie/data/numerotations_linux.qgz
avant chargement myProject = /home/fred/Documents/technic/source/voirie/data/numerotations_linux.qgz
après chargement myProject = /home/fred/Documents/technic/source/voirie/data/numerotations_linux.qgz
layoutName = TADebugNumero
coverageLayer = lj_demande_numero_parcelle
atlasFilter = 
outputName = query producing unique value
myLayout = <qgis._core.QgsPrintLayout object at 0x7f5f5af509d8>
myLayout = <qgis._core.QgsPrintLayout object at 0x7f5f5af50ee8>
myLayout = <qgis._core.QgsPrintLayout object at 0x7f5f5af65048>
myLayout = <qgis._core.QgsPrintLayout object at 0x7f5f5af659d8>
myLayout = <qgis._core.QgsPrintLayout object at 0x7f5f5af65678>
myLayout = <qgis._core.QgsPrintLayout object at 0x7f5f5af65438>
myLayout = <qgis._core.QgsPrintLayout object at 0x7f5f5af653a8>
myLayout = <qgis._core.QgsPrintLayout object at 0x7f5f5af65288>
myLayout = <qgi